# Create DSM with fewer holes from South Cascade 2019 (merged) lidar dataset
* See section "May 25 23 redo to reduce number of holes in SCG DSM"

In [3]:
laz_example_fn = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/data/20190506_SOUTH_CASCADE_DATASET/tiles/649000_5351000.laz"

laz_merged_fn = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/data/20190506_SOUTH_CASCADE_DATASET/merged_tiles.laz"



## deduplication

In [42]:
ulx, uly, lrx, lry = 638900, 5362900, 652400, 5349700

# ulx, uly, lrx, lry = 638900, 5362900, 639100, 5362700

pdal_bounds = f"([{ulx}, {lrx}], [{lry}, {uly}])"

In [36]:
laz_merged_deduplicated_fn = "merged_tiles_deduplicated.laz"
!pdal translate -i "$laz_merged_fn" -o "$laz_merged_deduplicated_fn" -f filters.sample --filters.sample.radius=0.001

In [44]:
dsm_merged_deduplicated_fn = "scg_merged_deduplicated_all.tif" #count_smalltest.tif"

# just "count" instead of "all"
!pdal translate -i "$laz_merged_deduplicated_fn" -o "$dsm_merged_deduplicated_fn" \
-f filters.reprojection --filters.reprojection.out_srs="$proj" \
-f filters.returns --filters.returns.groups="first,only" \
-f filters.outlier \
-f filters.range --filters.range.limits="Classification![7:7]" --writers.gdal.resolution="$res" \
-f filters.crop --filters.crop.bounds="$pdal_bounds" \
--writers.gdal.output_type="count" --writers.gdal.data_type="float32" \
--writers.gdal.gdalopts="COMPRESS=LZW,TILED=YES,BIGTIFF=IF_SAFER" 


PDAL: writers.gdal: Grid width out of range.



## TODO try again with fixed round number bounds (matching point2dem) to get the pixels right?

In [2]:
!pdal info $laz_example_fn

{
  "file_size": 6306159,
  "filename": "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/data/20190506_SOUTH_CASCADE_DATASET/tiles/649000_5351000.laz",
  "now": "2023-01-31T20:57:43-0800",
  "pdal_version": "2.4.3 (git-version: cd43bf)",
  "reader": "readers.las",
  "stats":
  {
    "bbox":
    {
      "EPSG:4326":
      {
        "bbox":
        {
          "maxx": -120.9768877,
          "maxy": 48.30387076,
          "maxz": 2279.75,
          "minx": -120.9914085,
          "miny": 48.29418206,
          "minz": 1455.84
        },
        "boundary": { "type": "Polygon", "coordinates": [ [ [ -120.991408533724581, 48.294430113676718, 1455.84 ], [ -120.991038005015341, 48.303870763963438, 1455.84 ], [ -120.976887680414578, 48.303622625644877, 2279.75 ], [ -120.977260814653263, 48.294182057114838, 2279.75 ], [ -120.991408533724581, 48.294430113676718, 1455.84 ] ] ] }
      },
      "native":
      {
        "bbox":
        {
          "maxx": 650024.99,
          "maxy": 5352024.99,
          "

In [34]:
laz_example_fn

'/mnt/1.0_TB_VOLUME/sethv/resdepth_all/data/20190506_SOUTH_CASCADE_DATASET/tiles/649000_5351000.laz'

In [9]:
laz_example_fn = "/mnt/1.0_TB_VOLUME/sethv/resdepth_all/data/20190506_SOUTH_CASCADE_DATASET/tiles/649000_5351000.laz"
dsm_example_fn = "scg_test_tile_dsm.tif"

res = 1.0
proj = "EPSG:32610"

# PDAL to create DSM at 1 meter posting in the selected CRS
# outlier filter may or may not be needed
# TODO check whether point cloud is classified re: noise points
# TODO not specifying bounds means pixel centers are not round numbers
# 6 bands of output (min max mean idw count stdev)
!pdal translate -i "$laz_example_fn" -o "$dsm_example_fn" \
-f filters.reprojection --filters.reprojection.out_srs="$proj" \
-f filters.returns --filters.returns.groups="first,only" \
-f filters.outlier \
-f filters.range --filters.range.limits="Classification![7:7]" --writers.gdal.resolution="$res" \
--writers.gdal.output_type="all" --writers.gdal.data_type="float32" \
--writers.gdal.gdalopts="COMPRESS=LZW,TILED=YES,BIGTIFF=IF_SAFER" 



In [10]:
dsm_merged_fn = "scg_merged_dsm.tif"

!pdal translate -i "$laz_merged_fn" -o "$dsm_merged_fn" \
-f filters.reprojection --filters.reprojection.out_srs="$proj" \
-f filters.returns --filters.returns.groups="first,only" \
-f filters.outlier \
-f filters.range --filters.range.limits="Classification![7:7]" --writers.gdal.resolution="$res" \
--writers.gdal.output_type="all" --writers.gdal.data_type="float32" \
--writers.gdal.gdalopts="COMPRESS=LZW,TILED=YES,BIGTIFF=IF_SAFER" 


In [ ]:
dsm_merged_point2dem_fn = "scg_merged_dsm_point2dem.tif"

In [21]:
# !point2dem --tr "$res" --t_srs "$proj" $laz_merged_fn -o "$dsm_merged_point2dem_fn"

# Just uses all the points from the lidar without any filtering by classification or outlier removal
!point2dem --tr 1.0 --t_srs EPSG:32610 merged_tiles.laz -o scg_merged_dsm_point2dem.tif

	--> Setting number of processing threads to: 4
Writing log info to: scg_merged_dsm_point2dem.tif-log-point2dem-01-31-2143-25591.txt
Writing temporary file: scg_merged_dsm_point2dem.tif-tmp-merged_tiles.tif
        --> [********************************************************] Complete!
The point cloud files must have an equal number of channels which must be 4 or 6 to be able to remove outliers.
Statistics: [********************************************************] Complete!
QuadTree: [**********************************************************] Complete!
	-- Starting DEM rasterization --
	--> DEM spacing: 1 pt/px
	             or: 1 px/pt
Creating output file that is Vector2(13468,13027) px.
Writing: scg_merged_dsm_point2dem.tif-DEM.tif
DEM: [***************************************************************] Complete!
Percentage of valid pixels: 59.0621%


In [22]:
!gdalinfo scg_merged_dsm_point2dem.tif-DEM.tif

Driver: GTiff/GeoTIFF
Files: scg_merged_dsm_point2dem.tif-DEM.tif
Size is 13468, 13027
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 10N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 10N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-123,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
            ID["E

In [23]:
!gdalinfo scg_merged_dsm.tif

Driver: GTiff/GeoTIFF
Files: scg_merged_dsm.tif
Size is 13099, 12996
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 10N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 10N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-123,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]],
     

In [ ]:
dem_align.py

In [ ]:
## crop to area around  643285.923,5355177.729
- would be v interesting but unlikely that orthoimage shows any useful texture here to see crown
- max. 1 meter crown based on 1 meter ish between pixels, 
- roughly 10 pts / m^2 means 0.5 meter grid should have points in each cell
- how to find the laz file that ocntains this area better to just pdal crop

In [33]:

!pdal translate -i "$laz_merged_fn" -o cropped_to_avalanches.laz \
-f filters.crop --filters.crop.distance 2000 --filters.crop.point "POINT(643940 5354860 1600)"

## May 25 23 redo to reduce number of holes in SCG DSM


In [ ]:
!pdal translate -i "$laz_merged_deduplicated_fn" -o "$dsm_merged_deduplicated_fn" \
-f filters.reprojection --filters.reprojection.out_srs="$proj" \
-f filters.returns --filters.returns.groups="first,only" \
-f filters.outlier \
-f filters.range --filters.range.limits="Classification![7:7]" --writers.gdal.resolution="$res" \
-f filters.crop --filters.crop.bounds="$pdal_bounds" \
--writers.gdal.output_type="count" --writers.gdal.data_type="float32" \
--writers.gdal.gdalopts="COMPRESS=LZW,TILED=YES,BIGTIFF=IF_SAFER" 

In [1]:
laz_merged_deduplicated_fn = "merged_tiles_deduplicated.laz"
laz_without_outliers_and_noise_fn = "scg_merged_deduplicated_filtered.laz"

# Filter out noise and outliers to get the desired points for point2dem to make DSM
# we don't use pdal to make DSMs because we want to give identical bounds
# to make stereo DSM and lidar DSM match with pixel centers, etc.
!pdal translate -i "$laz_merged_deduplicated_fn" -o "$laz_without_outliers_and_noise_fn" \
-f filters.returns --filters.returns.groups="first,only" \
-f filters.outlier \
-f filters.range --filters.range.limits="Classification![7:7]"

In [2]:
ulx=639750
uly=5359750
lrx=648750
lry=5350750

projwin_mediumsize=f"{ulx} {uly} {lrx} {lry}"

output_res = 1.0
proj = "EPSG:32610"
lidar_dem_prefix = f"filtered_scg_merged_lidar_dsm_{output_res}m"

!point2dem --tr $output_res --t_srs "$proj" \
    --t_projwin $projwin_mediumsize \
    --search-radius-factor 2 \
    $laz_without_outliers_and_noise_fn -o $lidar_dem_prefix

	--> Setting number of processing threads to: 4
Cropping to (Origin: (639750, 5.35075e+06) width: 9000 height: 9000) pt. 
Writing log info to: filtered_scg_merged_lidar_dsm_1.0m-log-point2dem-05-25-1526-13475.txt
Writing temporary file: filtered_scg_merged_lidar_dsm_1.0m-tmp-scg_merged_deduplicated_filtered.tif
        --> [********************************************************] Complete!
The point cloud files must have an equal number of channels which must be 4 or 6 to be able to remove outliers.
Statistics: [********************************************************] Complete!
QuadTree: [**********************************************************] Complete!
	-- Starting DEM rasterization --
	--> DEM spacing: 1 pt/px
	             or: 1 px/pt
Creating output file that is Vector2(9001,9001) px.
Writing: filtered_scg_merged_lidar_dsm_1.0m-DEM.tif
DEM: [**..................................................................] 4%Warning: Cache size (1056 MB) is larger than the requested maxim